In [1]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import Aer, transpile, assemble
import math
import random
import numpy as np
from scipy.optimize import minimize


## This circuit will attempt to solve 4x4 matrix (4 variables in the linear equations), with $|b> = H^{\otimes n} |0>$ and A as a linear combination of c_1 * I + c_2 * Z:
(b is the result vector, H is the hadamard gate and |0> is the base state (1 0)) where n = 2
(A is the matrix where A|x> = |b>, I is identity matrix and Z is the pauli Z)

In [2]:
def ansatz(circuit, qubits, params):
    
    circuit.ry(params[0][0], qubits[0])
    circuit.ry(params[0][1], qubits[1])

    circuit.cz(qubits[0], qubits[1])

    circuit.ry(params[1][0], qubits[0])
    circuit.ry(params[1][1], qubits[1])

circuit = QuantumCircuit(2)
ansatz(circuit, [0, 1], [[1.0, 1.0], [1.0, 1.0]])
circuit.draw()

┌───────┐   ┌───────┐
q_0: ┤ Ry(1) ├─■─┤ Ry(1) ├
     ├───────┤ │ ├───────┤
q_1: ┤ Ry(1) ├─■─┤ Ry(1) ├
     └───────┘   └───────┘

### Hadamard test for $<\Phi|\Phi>$

In [3]:
def gate(circuit, qubit, gate):
    if gate == 1:
        circuit.cx(0, qubit)
    elif gate == 2:
        circuit.cy(0, qubit)
    elif gate == 3:
        circuit.cz(0, qubit)

def had_test(circuit, qubits, params, gates):
    circuit.h(0)
    ansatz(circuit, qubits, params)
    for i in range(0, len(gates[0])):
        gate(circuit, qubits[i], gates[0][i])
    for i in range(0, len(gates[1])):
        gate(circuit, qubits[i], gates[1][i])
    circuit.h(0)

circuit = QuantumCircuit(3)
had_test(circuit, [1, 2], [[1, 1], [1, 1]], [[0, 1], [0, 0]])
circuit.draw()


┌───┐                   ┌───┐
q_0: ──┤ H ├────────────────■──┤ H ├
     ┌─┴───┴─┐   ┌───────┐  │  └───┘
q_1: ┤ Ry(1) ├─■─┤ Ry(1) ├──┼───────
     ├───────┤ │ ├───────┤┌─┴─┐     
q_2: ┤ Ry(1) ├─■─┤ Ry(1) ├┤ X ├─────
     └───────┘   └───────┘└───┘

### A new ansatz, but this time for $|<b|\Phi>|^2$
### Following the equation: 
### $$|\langle b | \Phi \rangle|^2 \ = \ \displaystyle\sum_{m} \displaystyle\sum_{n} c_m c_n \langle 0 | U^{\dagger} A_n V(k) | 0 \rangle \langle 0 | U^{\dagger} A_m V(k) | 0 \rangle$$

In [4]:
def control_ansatz(circuit, qubits, params):

    circuit.cry(params[0][0], 0, qubits[0])
    circuit.cry(params[0][1], 0, qubits[1])

    circuit.ccx(0, qubits[1], 3)
    circuit.cz(qubits[0], 3)
    circuit.ccx(0, qubits[1], 3)

    circuit.cry(params[1][0], 0, qubits[0])
    circuit.cry(params[1][1], 0, qubits[1])

circuit = QuantumCircuit(4)
control_ansatz(circuit, [1, 2], [[1, 1], [1, 1]])
circuit.draw()

q_0: ────■────────■──────■───────■──────■────────■────
     ┌───┴───┐    │      │       │  ┌───┴───┐    │    
q_1: ┤ Ry(1) ├────┼──────┼───■───┼──┤ Ry(1) ├────┼────
     └───────┘┌───┴───┐  │   │   │  └───────┘┌───┴───┐
q_2: ─────────┤ Ry(1) ├──■───┼───■───────────┤ Ry(1) ├
              └───────┘┌─┴─┐ │ ┌─┴─┐         └───────┘
q_3: ──────────────────┤ X ├─■─┤ X ├──────────────────
                       └───┘   └───┘

### Create the |b> value from the circuit, following $|b> = H^{\otimes n} |0>$

In [5]:
def b_value(circuit, qubits):
    # Create ONE new auxiliary qubits at postion 0
    # we now have 3 qubits: q_0 (auxiliary) q_1 and q_2 (standard)
    # Apply control H gate from q_0 to q_1 and q_2 consecutively

    circuit.ch(0, qubits[0])
    circuit.ch(0, qubits[1])

circuit = QuantumCircuit(3)
b_value(circuit, [1,2])
circuit.draw()

q_0: ──■────■──
     ┌─┴─┐  │  
q_1: ┤ H ├──┼──
     └───┘┌─┴─┐
q_2: ─────┤ H ├
          └───┘

### Hadamard test for $|\langle b | \Phi \rangle|^2 $

In [8]:
def special_had_test(circuit, qubits, params, gates):
    circuit.h(0)
    control_ansatz(circuit, qubits, params)
    for i in range(0, len(gates)):
        gate(circuit, qubits[i], gates[i])
    b_value(circuit, qubits)
    circuit.h(0)

circuit = QuantumCircuit(4)
special_had_test(circuit, [1, 2], [[1, 1], [1, 1]], [0, 3])
circuit.draw(fold=-1)

┌───┐                                                              ┌───┐
q_0: ┤ H ├────■────────■──────■───────■──────■────────■─────■───■────■──┤ H ├
     └───┘┌───┴───┐    │      │       │  ┌───┴───┐    │     │ ┌─┴─┐  │  └───┘
q_1: ─────┤ Ry(1) ├────┼──────┼───■───┼──┤ Ry(1) ├────┼─────┼─┤ H ├──┼───────
          └───────┘┌───┴───┐  │   │   │  └───────┘┌───┴───┐ │ └───┘┌─┴─┐     
q_2: ──────────────┤ Ry(1) ├──■───┼───■───────────┤ Ry(1) ├─■──────┤ H ├─────
                   └───────┘┌─┴─┐ │ ┌─┴─┐         └───────┘        └───┘     
q_3: ───────────────────────┤ X ├─■─┤ X ├────────────────────────────────────
                            └───┘   └───┘

### Cost function: we take this for granted for now

In [52]:
def cost_function(params, gates, coeffs):

    qubits = [1, 2]
    params = [params[0:2], params[2:4]]

    sum_1 = 0
    for i in range(0, len(gates)):
        for j in range(0, len(gates)):

            qr = QuantumRegister(4)
            cr = ClassicalRegister(4)
            circuit = QuantumCircuit(qr, cr)
            backend = Aer.get_backend('aer_simulator')

            had_test(circuit, qubits, params, [gates[i], gates[j]])

            circuit.save_statevector()
            res = backend.run(assemble(transpile(circuit, backend))).result()
            output = np.real(res.get_statevector(circuit, decimals=100))

            sum_i = 0
            for l in range (0, len(output)):
                if (l%2 == 1):
                    sum_i += output[l]**2 
            sum_1 += coeffs[i] * coeffs[j] * (1 - (2 * sum_i))

    sum_2 = 0
    for i in range(0, len(gates)):
        for j in range(0, len(gates)):

            prod = 1
            for k in range(0, 2):
                
                qr = QuantumRegister(4)
                cr = ClassicalRegister(4)
                circuit = QuantumCircuit(qr, cr)
                backend = Aer.get_backend('aer_simulator')

                special_had_test(circuit, qubits, params, gates[i if k == 0 else j])

                circuit.save_statevector()
                res = backend.run(assemble(transpile(circuit, backend))).result()
                output = np.real(res.get_statevector(circuit, decimals=100))

                sum_i = 0
                for l in range (0, len(output)):
                    if (l%2 == 1):
                        sum_i += output[l]**2 
                prod *= (1 - (2 * sum_i))
            
            sum_2 += coeffs[i] * coeffs[j] * prod
    
    ret = 1 - float(sum_2 / sum_1)
    print(ret)
    return ret


## Matrix A's unitary decomposition

In [ ]:
from itertools import product
import numpy as np

from qiskit.quantum_info import Operator
from qiskit.circuit import library

def make_square(matrix):
    # Pad with 0 to make square matrix
    if matrix.shape[0] != matrix.shape[1]:
        if matrix.shape[0] > matrix.shape[1]:
            # Pad columns
            pad_width = [(0, 0), (0, matrix.shape[0] - matrix.shape[1])]
        else:
            # Pad rows
            pad_width = [(0, matrix.shape[1] - matrix.shape[0]), (0, 0)]
        matrix = np.pad(matrix, pad_width)    
    return matrix

def get_pauli_bases(dimension):
    pauli = {
        'x': Operator(library.XGate().to_matrix()),
        'y': Operator(library.YGate().to_matrix()),
        'z': Operator(library.ZGate().to_matrix()),
        'i': Operator(library.IGate().to_matrix())
    }

    bases = {}
    # Creates bases matrix in dimension mentioned in parameter
    # For dimension 1, bases = {I, X, Y, Z}
    # For dimension 2, bases = {II, IX, IY, IZ, XX, XY, XZ, YY, YZ, ZZ}
    if dimension == 1:
        return pauli
    else:
        for permutation in product(*[list(pauli.keys())]*dimension):
            permutation = "".join(permutation)
            bases[permutation] = pauli[permutation[0]]
            for idx in range(1, len(permutation)):
                bases[permutation] = bases[permutation].tensor(pauli[permutation[idx]])
        return bases

def linear_combination_pauli(matrix):
    matrix = make_square(matrix)
    matrix_len = matrix.shape[0]
    
    # Assuming matrix dimension is in power of 2
    nqubits = int(np.log2(matrix_len))
    
    bases = get_pauli_bases(nqubits)
    decomposition = {}
    for base, base_matrix in bases.items():
        decomposition[base] = np.trace(np.dot(base_matrix, matrix)) / matrix_len
        
    return decomposition, bases

def validate_decomposition(decompositon, bases, original):
    created = sum(coeff*matrix.data for coeff, matrix in zip(decomposition.values(), bases.values()))
    created = np.around(created, 3)
    return (created == original).all()

In [39]:
gates = [[0, 0], [0, 3]]
coeffs = [0.55, 0.45]

out = minimize(cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 4)], args=(gates, coeffs), method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:2], out['x'][2:4]]
circuit = QuantumCircuit(2, 2)
ansatz(circuit, [0, 1], out_f)
circuit.save_statevector()

backend = Aer.get_backend('aer_simulator')
res = backend.run(assemble(transpile(circuit, backend))).result()
output = res.get_statevector(circuit, decimals=10)
print(output)

a1 = coeffs[1]*np.array([[1,0,0,0], [0,1,0,0], [0,0,-1,0], [0,0,0,-1]])
a2 = coeffs[0]*np.array([[1,0,0,0], [0,1,0,0], [0,0,1,0], [0,0,0,1]])
a3 = np.add(a1, a2)

b = np.array([float(1/2),float(1/2),float(1/2),float(1/2)])
print((b.dot(a3.dot(output)/(np.linalg.norm(a3.dot(output)))))**2)


0.8382319408508219
0.9957533474024632
0.8809225012830304
0.9969461020127046
0.8178280458569438
0.36736655061726364
0.4676830708832691
0.35186340923229553
0.2546206334968265
0.4702772498876502
0.2602386631364416
0.35462381960083844
0.2746081322157319
0.49898032602110287
0.34000548559557253
0.28296543573331234
0.19704958979163145
0.14620654000862165
0.11198067348558449
0.17967996672464048
0.13424746120063147
0.19068187525925684
0.03932666112338579
0.05878615960136746
0.02988389856975804
0.0879323755761876
0.031812806169415064
0.004361089012968056
0.06136616092443847
0.010947064528401329
0.045740317693444776
0.013648667765333244
0.001984431904140549
0.010210050144095817
0.0024553670100863068
0.00893814585574304
0.0030627129575278023
0.0018091411471450325
0.0014990169080061344
0.0014333022195515133
0.0015421387894124505
0.001415973374450874
0.0010667651272447953
0.0014094646724565063
0.000986455496191474
0.0009475277696339157
0.0009970263660366019
0.0009331287284553014
0.000884318383515103

In [9]:
# X gate test

def b_value(circuit, qubits):
    circuit.ch(0, qubits[0])

gates = [[0, 3], [1, 1]]
coeffs = [0.5, 0.5]

out = minimize(cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 4)], args=(gates, coeffs), method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:2], out['x'][2:4]]
circuit = QuantumCircuit(2, 2)
ansatz(circuit, [0, 1], out_f)
circuit.save_statevector()

backend = Aer.get_backend('aer_simulator')
res = backend.run(assemble(transpile(circuit, backend))).result()
output = res.get_statevector(circuit, decimals=10)
print(output)

a1 = coeffs[0]*np.array([[1,0,0,0], [0,1,0,0], [0,0,-1,0], [0,0,0,-1]])
a2 = coeffs[1]*np.array([[0,0,0,1], [0,0,1,0], [0,1,0,0], [1,0,0,0]])
a3 = np.add(a1, a2)

b = np.array([float(1/np.sqrt(2)),float(1/np.sqrt(2)),0,0])
print((b.dot(a3.dot(output)/(np.linalg.norm(a3.dot(output)))))**2)


0.6828207642106855
0.976630941451887
0.8695422165342892
0.969301372639352
0.7991459103542358
0.14283208976551176
0.18682384251384798
0.1855066092545784
0.14411326193980856
0.24357799450348294
0.15108962921009128
0.13354361275288873
0.2038565694943758
0.09864703376654649
0.04109552504821179
0.04306833177479086
0.031964566117187676
0.03914053341020818
0.02425109260901115
0.012586838923689836
0.021445900526714357
0.009653864012817315
0.002373932420235536
0.006710527201050032
0.0089027790989602
0.0009468362781509532
0.0007432348110703035
0.0021062436551262387
0.00023664482648599883
0.0007710178323215855
0.0003241852644091381
0.0004847213912528847
0.0010395999255029542
0.0001577294016221753
4.5862871920765436e-05
3.1750990440948534e-05
3.0321014483858377e-05
0.0001841391115203761
1.558703393134131e-05
1.6106284779326963e-05
1.7144845067429415e-05
4.699323849055581e-05
2.6093121173076383e-05
5.126676087430226e-06
1.9346120836760683e-05
7.774964205120583e-06
1.2001080234247397e-05
1.268835158

In [23]:
# Y gate test

def b_value(circuit, qubits):
    circuit.cy(0, qubits[0])

gates = [[1, 1]]
coeffs = [1]

out = minimize(cost_function, x0=[float(random.randint(0,3000))/1000 for i in range(0, 4)], args=(gates, coeffs), method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:2], out['x'][2:4]]
circuit = QuantumCircuit(2, 2)
ansatz(circuit, [0, 1], out_f)
circuit.save_statevector()

backend = Aer.get_backend('aer_simulator')
res = backend.run(assemble(transpile(circuit, backend))).result()
output = res.get_statevector(circuit, decimals=10)
print(output)

a = np.array([[0, -1.j, 0, 0], [1.j, 0, 0, 0], [0, 0, 0, -1.j], [0, 0, 1.j, 0]])
b = np.array([0,1.j,0,0])
print((b.dot(a.dot(output)/(np.linalg.norm(a.dot(output)))))**2)

0.8544936413189912
0.496801210525343
0.4449358478633524
0.5783386598219695
0.09284039661420007
0.11512791716434845
0.11958099344653206
0.10284155136579998
0.18175300392098936
0.04057189708019204
0.05615095388739466
0.04515144696370166
0.03096980503980773
0.05379784485335959
0.03588951653146877
0.015489614576627897
0.00978837779249564
0.002979404800515062
0.0016850437290598652
0.0025199605191957852
0.0001734783478941626
0.008990593766880162
0.005041927363679477
0.0011841162120607152
0.0008177071308396178
0.00013394544443923184
0.00013677350160323787
0.0004261514577742487
0.00010330806441227747
0.00048412057292068766
0.00011803503373586377
0.00023306153199653679
6.982868947180698e-06
0.00012007670877611787
3.9356345419627736e-05
6.741867881854269e-06
5.776524537415284e-05
4.6934904388562515e-06
8.846097866666014e-06
7.4446316372656085e-06
1.6024103251854882e-05
1.3407734451575948e-06
1.6003546839948868e-06
1.5202047571882815e-06
4.560806021736141e-07
3.046969218045703e-07
6.8769710270810

Small test to find out if it calculates it correctly.

In [45]:
def b_value(circuit, qubits):
    # Create ONE new auxiliary qubits at postion 0
    # we now have 3 qubits: q_0 (auxiliary) q_1 and q_2 (standard)
    # Apply control H gate from q_0 to q_1 and q_2 consecutively

    circuit.ch(0, qubits[0])
    circuit.ch(0, qubits[1])


gates = [[0, 0], [3, 0]]
coeffs = [0.75, 0.25]

out = minimize(cost_function, x0=[float(random.randint(0,1000))/1000 for i in range(0, 4)], args=(gates, coeffs), method="COBYLA", options={'maxiter':200})
print(out)

out_f = [out['x'][0:2], out['x'][2:4]]
circuit = QuantumCircuit(2, 2)
ansatz(circuit, [0, 1], out_f)
circuit.save_statevector()

backend = Aer.get_backend('aer_simulator')
res = backend.run(assemble(transpile(circuit, backend))).result()
output = res.get_statevector(circuit, decimals=10)
print(output)

#a1 = coeffs[0]*np.array([[1,0,0,0], [0,1,0,0], [0,0,-1,0], [0,0,0,-1]])
#a2 = coeffs[1]*np.array([[0,0,0,1], [0,0,1,0], [0,1,0,0], [1,0,0,0]])
#a3 = np.add(a1, a2)

a3 = np.array([[1,0,0,0],[0,0.5,0,0],[0,0,1,0],[0,0,0,0.5]])

b = np.array([1/2,1/2,1/2,1/2])
print((b.dot(a3.dot(output)/(np.linalg.norm(a3.dot(output)))))**2)

0.44096989747330906
0.21567147030067724
0.34545379308741186
0.08362877558939252
0.08324365540155909
0.8927809593853822
0.065537666587562
0.11519591470219948
0.23618405793191888
0.12985936177631452
0.04446609332907747
0.09089123742717031
0.049867484658262407
0.06691381697225895
0.05305671188923766
0.016376646170864873
0.010151387255341482
0.012940480535288512
0.003768424276980653
0.004457220920778271
0.0035050760548225313
0.006220352488872094
0.00592439178918569
0.001519914752764806
0.0002379169593250685
0.0012500797597022606
0.0006093243468380249
0.004109656961467922
0.0020611014512319503
0.0002834654198198283
0.00020554524086691117
0.00016933194834822807
0.00015022714717638497
0.0003486710298347129
5.0081030500859214e-05
0.0002768625446145645
2.0132136998363137e-05
2.6244892411098064e-05
4.964698600251616e-05
5.7642836732596514e-05
1.3953656604726028e-05
1.299218283579151e-05
1.5271630726010343e-05
2.4669278915534498e-05
2.3783828902446125e-05
3.3024895592648207e-06
3.539509487726633e

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8e107b99-ec0e-43d9-aa18-8c6ee91759cf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>